In [24]:
from ftplib import FTP
import copy

import numpy as np

from astropy.coordinates import Angle
from astropy.time import Time, TimeDelta
from astropy import units as u
# для astropy в режиме offline
from astropy.utils import iers
iers.conf.auto_download = True 

from secret_key import login, password

In [25]:
"""
N1 – 194.149.67.97 (от 20° до 42°)
N2 – 194.149.67.38 (от -9° до 20°)
"""

con = FTP('194.149.67.97', login, password)

In [26]:
data = []
con.dir(data.append)

In [27]:
file_list = con.nlst('*.pnt')

In [28]:
data = []
def read_pnt_header(line):
    data.append(line.decode("utf-8"))

In [29]:
def get_time_begin_and_end(header):
    if 'UTC' in header['date_begin']:
        # begin
        day, month, year = header['date_begin'][2].split('.')
        hour, minute, second = header['time_begin'][2].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_start = Time(isot_time,
                          format='isot', scale='utc', location=('37.36d', '54.50d'),precision=7)
        # end
        day, month, year = header['date_end'][0].split('.')
        hour, minute, second = header['time_end'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_end = Time(isot_time,
                        format='isot', scale='utc', location=('37.36d', '54.50d'),precision=7)
    else:
        # begin
        day, month, year = header['date_begin'][0].split('.')
        hour, minute, second = header['time_begin'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_start = Time(isot_time,
                          format='isot', scale='utc', location=('37.36d', '54.50d'),precision=7)
        time_start -= 4*u.hour
        # end
        day, month, year = header['date_end'][0].split('.')
        hour, minute, second = header['time_end'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second 
        )
        time_end = Time(isot_time,
                        format='isot', scale='utc', location=('37.36d', '54.50d'),precision=7)
        time_end -= 4*u.hour
        
    return time_start, time_end


In [30]:
try:
    con.retrbinary('RETR ' + file_list[0], callback=read_pnt_header, blocksize=493)
except UnicodeDecodeError:
    print('Read was done!')
except ConnectionResetError:
    print('!!! Connection was aborted !!!')

Read was done!


In [31]:
header = {}
for line in data[0].split('\n'):
    a, *b = line.split()
    header[a] = b

In [32]:
header

{'alpha': ['alpha'],
 'date_begin': ['31.03.2019', 'UTC', '31.03.2019'],
 'date_end': ['31.03.2019'],
 'delta': ['delta'],
 'fbands': ['109.20751953125',
  '109.62255859375',
  '110.03759765625',
  '110.45263671875',
  '110.86767578125',
  '111.287597656'],
 'fcentral': ['110.25'],
 'modulus': ['1', '2', '3', '4', '5', '6'],
 'nbands': ['6'],
 'npoints': ['36018'],
 'numpar': ['16'],
 'source': ['source'],
 'time_begin': ['23:00:00', 'UTC', '19:00:00'],
 'time_end': ['23:59:59'],
 'tresolution': ['99.9424'],
 'wb_total': ['2.5'],
 'wbands': ['0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4248046875']}

In [33]:
t_b, t_e = get_time_begin_and_end(header)

In [34]:
print(t_b, t_e)

2019-03-31T19:00:00.0000000 2019-03-31T23:59:59.0000000


In [35]:
t_b.utc

<Time object: scale='utc' format='isot' value=2019-03-31T19:00:00.0000000>

In [40]:
# startime_begin = (t_b.sidereal_time('apparent'))
# startime_end = (t_e.sidereal_time('apparent'))

In [37]:
def my_sidereal_time(time):
    t2000 = Time('2000-01-01T00:00:00', format='isot', scale='utc', precision=7)
    t = time.jd - t2000.jd - 1
    t /= 36525
    s0 = 6 + 41 / 60.0 + 50.55 / 3600.0 + 8640184 / 3600.0 * t + 0.093104 / 3600.0 * t * t - 6.27 / 3600.0 * (1e-6) * t * t * t
    t_culm = (time.datetime.hour*u.hour 
          + time.datetime.minute*u.minute
          + time.datetime.second*u.second
          + time.datetime.microsecond*u.microsecond
             )
    alambda = 2 + 30/60.0 + 34/3600.0
    cnst = 2.7379093e-3
    s_culm = s0 + (cnst + 1) * t_culm.value + alambda
    delta_lucha = 0.89
    fi = 0.956829
    be = 0.008436
    delt = delta_lucha
    culm = s_culm
    aa = ((np.sin(fi))**2) * ((np.cos(be))**2) + ((np.cos(fi))**2)
    bb = 2 * np.sin(fi) * np.cos(be) * np.sin(delt)
    cc = ((np.sin(delt))**2) - ((np.cos(fi))**2)

    x = (bb + np.abs(bb) - 4 * aa * cc) / (2 * aa)
    y = x * np.sin(be) / np.cos(delt)
    z = np.sqrt(1 - y * y)

    t = y / z
    alf = culm * np.pi / 12 + np.arctan(t)
    
    rs = 4.8481368e-6
    am = 46.1 * rs
    an = 20.4 * rs

    alfa1 = copy.copy(alf)
    delta1 = copy.copy(delt)
    alfa = copy.copy(alf)

    for i in range(2):
        alf_res = alfa - (am + an * np.sin(alfa1) * np.tan(delta1)) * t
        delt_res = delt - an * np.cos(alfa1) * t
        alfa1 = (alf_res + alfa) / 2
        delta1 = (delt_res + delt) / 2

    alf_res /= (15 * rs);
    delt_res /= rs;

    alf_res /= 3600;
    while alf_res >= 24:
        alf_res -= 24

    while alf_res < 0:
        alf_res += 24

    print(Angle(alf_res*u.hourangle))
    """
    if (realSeconds): {
    *realSeconds = alf_res * 3600;
    }
    """
    delt_res /= 3600;
    while delt_res >= 360:
        delt_res -= 360

    while delt_res < 0:
        delt_res += 360
    print(Angle(delt_res*u.hourangle))

In [38]:
my_sidereal_time(t_b)

10h09m23.9252s
50h59m35.8539s


In [42]:
my_sidereal_time(t_e - 4*u.hour)

11h09m42.6367s
50h59m35.8719s


In [152]:
iers.IERS_A_URL = 'http://toshi.nofs.navy.mil/ser7/finals2000A.all'
import astroplan

In [156]:
astroplan.test()

ERROR: usage: ipykernel_launcher.py [options] [file_or_dir] [file_or_dir] [...]
ipykernel_launcher.py: error: unrecognized arguments: --remote-data=none
  inifile: None
  rootdir: F:\YandexDisk\3.Programing\work\BSA\BSA3\bsa_reader



4